# Analiza temperatur v Sloveniji


Ta projekt izvaja zajem in obdelavo temperaturnih podatkov za 10 slovenskih mest s pomočjo Open-Meteo API-ja. Cilj je analizirati temperaturno zgodovino (2000-2025) po mestih, letnih časih in regijah ter identificirati podnebne trende.

Projektna naloga zajema:
- Pridobivanje zgodovinskih podatkov o temperaturah iz Open-Meteo API-ja
- Shranjevanje podatkov za zadnjih 25 let za 10 slovenskih mest
- Organizacija podatkov v mapah za nadaljno analizo

## Viri podatkov
- Open-Meteo API za zgodovinske vremenske podatke
- Geodetska uprava RS za koordinate mest
- ARSO za validacijo podatkov

## Cilj projekta
Analizirati temperaturno zgodovino (2000-2025) po mestih, letnih časih in regijah ter identificirati podnebne trende.

## Uvoz knjižnic

V tem delu uvozimo vse potrebne knjižnice

In [1]:
import requests                 # Za komunikacijo z API
import pandas as pd             # Za delo s podatki
import os                       # Za datoteke
from datetime import datetime   # Za datume
from tqdm import tqdm           # Za progress bar

print("✅ Vse knjižnice so uspešno naložene")

✅ Vse knjižnice so uspešno naložene


## Konfiguracija projekta

Nastavimo osnovne parametre za pridobivanje podatkov:
- API naslov za zgodovinske podatke
- Časovno obdobje (zadnjih 25 let)
- Mapa za shranjevanje rezultatov

In [2]:
# API Naslov
API_NASLOV = "https://archive-api.open-meteo.com/v1/archive"
STEVILO_LET = 25
CASOVNI_PAS = "Europe/Berlin"

# Mapa s podatki
IMENIK_PODATKOV = "podatki"
MAX_CAKANJE = 15  # v sekundah

## Izbrani kraji

Analizirali bomo 10 mest iz vseh delov Slovenije z njihovimi koordinatami:

In [3]:
kraji = {
    # Primorska
    "Koper": (45.5481, 13.7302),
    "Nova_Gorica": (45.9558, 13.6433),
    
    # Gorenjska
    "Bled": (46.3692, 14.1136),
    
    # Osrednja Slovenija
    "Ljubljana": (46.0569, 14.5058),
    "Celje": (46.2309, 15.2604),
    
    # Podravje
    "Maribor": (46.5547, 15.6459),
    
    # Dolenjska
    "Novo_mesto": (45.8000, 15.1667),
    "Krško": (45.9500, 15.4833),
    
    # Pomurje
    "Murska_Sobota": (46.6586, 16.1594),
    
    # Koroška
    "Slovenj_Gradec": (46.5100, 15.0800)
}

## Pridobivanje podatkov iz API-ja

Funkcija `pridobi_podatke` bo poslala zahtevo na Open-Meteo API za pridobitev temperaturne zgodovine za lokacijo in časovno obdobje. Te podatke bo vrnila v JSON formatu. Nastavimo še parametre:
   - `latitude` : Zemljepisna širina
   - `longitude` : Zemljepisna dolžina
   - `start_date` : Začetni datum (YYYY-MM-DD)
   - `end_date` : Končni datum (YYYY-MM-DD)
   - `temperature_2m_mean` : Povprečna dnevna temperatura

In [4]:
def pridobi_podatke(latitude, longitude, start_date, end_date):
    
    parametri = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "temperature_2m_mean",
        "timezone": "Europe/Berlin"
    }
    try:
        odgovor = requests.get(API_NASLOV, params=parametri, timeout=MAX_CAKANJE)
        return odgovor.json()
    except Exception as e:
        return None


## Shranjevanje podatkov

Podatke bomo organizirali v hierarhični strukturi:

```text
Projektna_naloga_UVP/
└── podatki/
    ├── Koper/
    │   ├── 2020/
    │   │   ├── 01.csv
    │   │   ├── 02.csv
    │   │   └── ...
    │   ├── 2021/
    │   │   ├── 01.csv
    │   │   └── ...
    │   └── ...
    │
    ├── Ljubljana/
    │   ├── 2020/
    │   │   ├── 01.csv
    │   │   └── ...
    │   └── ...
    │
    ├── Maribor/
    │   ├── 2020/
    │   │   ├── 01.csv
    │   │   └── ...
    │   └── ...
    │
    └── ostali_kraji/
        ├── Bled/
        ├── Celje/
        └── ...
```

## Funkcija za shranjevanje podatkov

Shrani temperaturno zgodovino v CSV datoteke po letih in mesecih

In [5]:
def shrani_podatke(kraj, podatki):
    
    try:
        # Ustvarimo DataFrame
        df = pd.DataFrame({
            "datum": podatki["daily"]["time"],
            "temperatura": podatki["daily"]["temperature_2m_mean"]
        })
        
        # Pretvorba datuma
        df['datum'] = pd.to_datetime(df['datum'])
        df['leto'] = df['datum'].dt.year
        df['mesec'] = df['datum'].dt.month
        
        # Mape po imenih
        os.makedirs(f"{IMENIK_PODATKOV}/{kraj}", exist_ok=True)
        
        # Mape po letih
        for (leto, mesec), skupina in df.groupby(['leto', 'mesec']):
            leto_mapa = f"{IMENIK_PODATKOV}/{kraj}/{leto}"
            os.makedirs(leto_mapa, exist_ok=True)
            # Mape po mesecih
            skupina.to_csv(f"{leto_mapa}/{mesec:02d}.csv", index=False)
        
    except Exception as e:
        pass


## Glavna zanka za obdelavo

Postopek pridobivanja podatkov:
1. Določimo časovno obdobje (zadnjih 25 let)
2. Za vsak kraj poženemo funkcijo `pridobi_podatke`
3. Uporabi funkcijo `shrani_podatke` za shranjevanje
4. Prek progress bara spremljamo napredek

In [7]:
def main():
    
    # Preverimo obstoj glavne mape
    os.makedirs("podatki", exist_ok=True)
    
    # Časovno obdobje (zadnjih 25 let)
    danes = datetime.now()
    end_date = danes.strftime("%Y-%m-%d")
    start_date = ("2000-01-01")
    
    # Progress bar:
    with tqdm(kraji.items(), desc="Pridobivanje podatkov") as pbar:
        for kraj, (lat, lon) in pbar:
            # Pridobimo podatke za trenutni kraj
            podatki = pridobi_podatke(lat, lon, start_date, end_date)
            # Shranimo podatke
            if podatki:
                shrani_podatke(kraj, podatki)
            # Posodobimo napredek
            pbar.update()

    # Ob zaključku pošljemo sporočilo
    sporocilo = f"""
    Uspešno shranjeni podatki:
    {'- Obdelanih krajev:':<25} {len(kraji):<23}
    {'- Časovno obdobje:':<25} {STEVILO_LET} let{'':<18}
    {'- Lokacija shranjevanja:':<25} {IMENIK_PODATKOV:<23}
    """
    print(sporocilo)

# Poženemo funkcijo
if __name__ == "__main__":
    main()


Pridobivanje podatkov: 100%|██████████| 10/10 [00:02<00:00,  4.72it/s]


    Uspešno shranjeni podatki:
    - Obdelanih krajev:       10                     
    - Časovno obdobje:        25 let                  
    - Lokacija shranjevanja:  podatki                
    
